In [2]:
import cv2 as cv
import cv2.aruco as aruco
import numpy as np
import os
import  depthai as dai

In [24]:
def getFrame(queue):
    #Get frame from the queue
    frame = queue.get()
    #convert the frmae to OpenCV format and return
    return frame.getCvFrame()

def getMonoCamera(pipeline, isLeft):
    #configure mono camera
    mono = pipeline.createMonoCamera()

 #set the Camera Resolution
    mono.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)
    if isLeft:
        mono.setBoardSocket(dai.CameraBoardSocket.LEFT)
    else:
        mono.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    return mono

if __name__ == '__main__':

    #define a pipeline
    pipeline = dai.Pipeline()

    #set up left and right cameras
    monoLeft = getMonoCamera(pipeline, isLeft = True)
    monoRight = getMonoCamera(pipeline, isLeft = False)

    #set output Xlink for left camera
    xoutLeft = pipeline.createXLinkOut()
    xoutLeft.setStreamName("left")

    #set output Xlink for right camera
    xoutRight = pipeline.createXLinkOut()
    xoutRight.setStreamName("right")

    #Attach cameras to output XLink 
    monoLeft.out.link(xoutLeft.input)
    monoRight.out.link(xoutRight.input)

    #pipeline is defined, now we can connect to the device
    with dai.Device(pipeline) as device:

        #get the output queues.
        leftQueue = device.getOutputQueue(name = 'left', maxSize=1)
        rightQueue = device.getOutputQueue(name = 'right', maxSize = 1)
        count=0
        while True:
            #get left Frame
            leftFrame = getFrame(leftQueue)
            #get right frame
            rightFrame = getFrame(rightQueue)

            cv.imshow('left', leftFrame)
            cv.imshow('right', rightFrame)

            #check for keyboard input
            key = cv.waitKey(1)
            if key == ord('q'):
                break # quit when q is pressed
            elif key == ord('p'):
                cv.imwrite(f'picture_left_{count}.png', leftFrame)
                cv.imwrite(f'picture_right_{count}.png', rightFrame)
                count += 1

In [3]:
# find the aruco marker in the image
def findArucoMarkers(img, markerSize =4, totalMarkers=250, draw=True):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    key = getattr(aruco, f'DICT_{markerSize}X{markerSize}_{totalMarkers}')
    arucoDict = aruco.Dictionary_get(key)
    arucoParam = aruco.DetectorParameters_create()
    # get the bounding box of the aruco markers
    bboxs, ids, rejected = aruco.detectMarkers(gray, arucoDict, parameters = arucoParam)
    return (bboxs)

In [7]:
# load the camera images captured seperated by 7.5 cm
img0 = cv.imread('./picture_left_0.png')
img1 = cv.imread('./picture_right_0.png')

#disparity equation d = basline(8cm) * focallength(1.636331765375964e+03)/(ul - ur)

baseline = 7.5
focal_length = 3.818394999478380e+02

actual_measurement = 136

#we use the right corners of aruco markers
bbox1= findArucoMarkers(img0)
bbox2 = findArucoMarkers(img1)

print(bbox1[0][0][3][0])
print(bbox2[0][0][3][0])

d = (baseline * focal_length)/(bbox1[0][0][3][0]-bbox2[0][0][3][0])
#d_y = (baseline * focal_length)/(bbox1[0][0][1][0]-bbox2[0][0][1][0])
print(f'Distance measured is {d} cm')
print(f'Actual distance is {actual_measurement} cm')
print(f'Error is {abs(d-actual_measurement)} cm')
# Distance measured is 208.28000000000003 cm
# Distance from disparity equation is 208.008 cm
# Error = 0.272 cm

353.0
331.0
Distance measured is 130.17255680039932 cm
Actual distance is 136 cm
Error is 5.82744319960068 cm
